In [1]:
import uproot as ut
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from uproot3_methods import TLorentzVectorArray
import vector
import awkward as ak
from tqdm import tqdm
%matplotlib inline

import numpy as np

from cutConfig import *

In [14]:
wpcut_map = ["no","loose","medium","tight"]
ptcut_list = [0]+jet_ptWP
btagcut_list = [0]+jet_btagWP
order_list = ["jet_ptRegressed","jet_btag"]
njet_list = range(1,7)

In [15]:
selection_list = []
for i,ptcut in enumerate(ptcut_list):
    for j,btagcut in enumerate(btagcut_list):
        for order in order_list:
            for njet in njet_list:
                selection_list.append([{"variable":order,"njets":njet,"btagcut":btagcut,"ptcut":ptcut,"njetcut":njet,"tag":f"{wpcut_map[i]} pt & {wpcut_map[j]} btag"}])

In [16]:
selection_info_list = [iterative_selection(branches,scheme)[0] for scheme in tqdm(selection_list)]

100%|██████████| 192/192 [29:55<00:00,  9.35s/it]


In [18]:
values_list = [ calculate_values(branches,mask=info["mask"],nsixb_selected=info["nsixb_selected"],
                                 nsixb_remaining=info["nsixb_remaining"],njets=info["njets"]) for info in selection_info_list ]

In [89]:
flatten_values = lambda value : np.array([ values[value] for values in values_list ])
fv = flatten_values

find_selection = lambda scheme : { i:selection for i,selection in enumerate(selection_list) if set(scheme.items()) == set(scheme.items()).intersection(set(selection[0].items()))}
fs = find_selection

print_index = lambda index : (print(selection_list[index]), print_values(values_list[index]))
pi = print_index

In [103]:
values = fv("signal_per_found") * fv("signal_eff") * fv("signal_avg_found")

for i in range(1,7):
    scheme_mask = fs({"njets":i,"ptcut":30})
    best_i,best_v = max( enumerate(values),key=lambda v : v[1] if v[0] in scheme_mask else -1 )

    pi(best_i)

[{'variable': 'jet_ptRegressed', 'njets': 1, 'btagcut': 0.7264, 'ptcut': 30, 'njetcut': 1, 'tag': 'medium pt & tight btag'}]
Event Efficiency:   0.99
Signal Efficiency:  0.99
Signal Purity:      0.36
Event Avg Found:    0.98 -> 98.29%
Signal Avg Found:   0.99 -> 99.10%
Event Total Found:  19.13%
Signal Total Found: 16.52%
[{'variable': 'jet_btag', 'njets': 2, 'btagcut': 0, 'ptcut': 30, 'njetcut': 2, 'tag': 'medium pt & no btag'}]
Event Efficiency:   1.0
Signal Efficiency:  1.00
Signal Purity:      0.36
Event Avg Found:    1.92 -> 95.95%
Signal Avg Found:   1.97 -> 98.48%
Event Total Found:  37.38%
Signal Total Found: 32.83%
[{'variable': 'jet_btag', 'njets': 3, 'btagcut': 0, 'ptcut': 30, 'njetcut': 3, 'tag': 'medium pt & no btag'}]
Event Efficiency:   0.99
Signal Efficiency:  1.00
Signal Purity:      0.36
Event Avg Found:    2.85 -> 95.06%
Signal Avg Found:   2.94 -> 98.11%
Event Total Found:  55.51%
Signal Total Found: 49.05%
[{'variable': 'jet_btag', 'njets': 4, 'btagcut': 0, 'ptcut'

In [94]:
scheme = {"tag":"no pt & tight btag","njets":1}
fs(scheme)

{36: [{'variable': 'jet_ptRegressed',
   'njets': 1,
   'btagcut': 0.7264,
   'ptcut': 0,
   'njetcut': 1,
   'tag': 'no pt & tight btag'}],
 42: [{'variable': 'jet_btag',
   'njets': 1,
   'btagcut': 0.7264,
   'ptcut': 0,
   'njetcut': 1,
   'tag': 'no pt & tight btag'}]}

In [95]:
pi(36);

[{'variable': 'jet_ptRegressed', 'njets': 1, 'btagcut': 0.7264, 'ptcut': 0, 'njetcut': 1, 'tag': 'no pt & tight btag'}]
Event Efficiency:   0.99
Signal Efficiency:  0.99
Signal Purity:      0.36
Event Avg Found:    0.98 -> 98.22%
Signal Avg Found:   0.99 -> 99.09%
Event Total Found:  19.12%
Signal Total Found: 16.52%
